# Note: Tunable Low Pass Filter Layer
This is just a note to self.  You should not have to read this.

In [6]:
    ''' Description:  This is a tunable 'reverse sigmoid' soft mask, which will get multiplied (element-wise)
                      by the (vertical columns of) frequency spectrum, to serve as a low-pass filter.
                      It has two parameters: the central frequency (from 0...1, scales with size of input)
                      and a steepness parameter (also between -1.0=flat and 1.0=vertical)
                      TODO: how do we best bound these to positive values?

        Purpose:      Much of the loss-reduction in the final network output gets bound up in damping high-frequency noise.
                      In theory we could let the network do this eventually, but this layer is to help converge faster.

        Note:          Guide at https://discuss.pytorch.org/t/how-to-define-a-new-layer-with-autograd/351
        
        TODO:          Make indices work with batch_size
    '''

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Mathematical Background Tunable 'Tophat' Sigmoid Filter (in freq domain)

$$ y(x) = \left[ 1+ \exp\left( {1+m\over 1+m+\epsilon}10\left({x\over L} - b - 0.5\right)  \right) \right]^{-1} $$
where 

* $m$ is the tunable 'steepness' parameter  (-1 = horizontal, 0 = 'normal',  1 = vertical)
* $b$ is the tunable 'center' parameter     (-0.5 = far left, 0 = middle, 0.5 = far right)
* $L$ is the number of frequency bins 
* $x = \{0..L\}$ is the frequency bin
* $\epsilon=0.1$ is there just to keep things from blowing up when $m=1$. 
* '10' is in there just because it looked about right. ;-) 

In [7]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

L = 8192      # number of frequency bins in my fft output
x = np.linspace(-1, L, num=100)     # num=100 is just for plotting. in code you want linspace(0,L-1,num=L)

def y(m, b):
    eps=0.1
    plt.figure(2)
    yvals = 1/(  1+np.exp( (1+m)/(1-m+eps)*10*(x/L-(b+0.5)) )  )
    plt.plot(x, yvals) 
    plt.ylim(0, 1)
    plt.show()

interactive_plot = interactive(y, m=(-1.0, 1.0,0.02), b=(-1, 1, 0.05))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='m', max=1.0, min=-1.0, step=0.02), FloatSlider(value=0.0, description='b', max=1.0, min=-1.0, step=0.05), Output(layout=Layout(height='350px'))), _dom_classes=('widget-interact',))

### PyTorch Code


In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class LowPassLayer(nn.Module):
    ''' Description:  This is a tunable 'reverse sigmoid' soft mask, which will get multiplied (element-wise)
                      by the (vertical columns of) frequency spectrum, to serve as a low-pass filter.
                      It has two parameters: the central frequency (from 0...1, scales with size of input)
                      and a steepness parameter (also between -1.0=flat and 1.0=vertical)
                      TODO: how do we best bound these to positive values?

        Purpose:      Much of the loss-reduction in the final network output gets bound up in damping high-frequency noise.
                      In theory we could let the network do this eventually, but this layer is to help converge faster.

        Note:          Guide at https://discuss.pytorch.org/t/how-to-define-a-new-layer-with-autograd/351
        
        TODO:          Make indices work with batch_size
    '''
    def __init__(self, bins):
        super(LowPassLayer, self).__init__()
        self.m = nn.Parameter(torch.zeros(1))      # steepness, tunable parameter, default is 0
        self.b = nn.Parameter(torch.zeros(1))      # center, tunable parameter, default is 0
        self.eps = 0.1
        self.bins = bins
        self.x = Variable(torch.linspace(0, bins-1, num=bins))
        
    def forward(self, in_stft):
        # given matrix A (n x m) and vector v (n elements), the operation we want is (A.T * v).T
        #   This method is also the fastest: https://stackoverflow.com/questions/18522216/multiplying-across-in-a-numpy-array

        m = self.m.expand_as(self.x)     # make it a 1-D vector of repeated numbers
        b = self.b.expand_as(self.x)
        mask = 1/(1+torch.exp( (1+m)/(1-m+self.eps)*10*(self.x/self.bins-(b+0.5)) )  ) 
        out_stft = torch.transpose( torch.mm( torch.transpose(in_stft), mask ) )
        return out_stft
